In [ ]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import joblib
from pathlib import Path

# lidhja me SQL Server (DW EUDWH)
conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=localhost,1433;"
    "DATABASE=EUDWH;"
    "UID=sa;"
    "PWD=YourStrong!Passw0rd;"
    "Encrypt=no;"
)

conn = pyodbc.connect(conn_str)
print("Lidhja me SQL Server u hap me sukses.")

query_population = """
SELECT
    c.geo_code,
    t.year,
    f.population_value
FROM dwh.fact_population f
JOIN dwh.dim_country c ON f.country_key = c.country_key
JOIN dwh.dim_time t ON f.time_key = t.time_key
WHERE c.geo_code IN ('DE', 'FR', 'IT', 'ES', 'PL', 'EU27_2020')
ORDER BY c.geo_code, t.year;
"""

df_pop = pd.read_sql(query_population, conn)
print(df_pop.head())
print(df_pop["geo_code"].value_counts())


query_migration = """
SELECT
    geo_code,
    year,
    value,
    citizenship
FROM dwh.fact_migration_stream
WHERE geo_code IN ('DE', 'FR', 'IT', 'ES', 'PL', 'EU27_2020')
ORDER BY geo_code, year;
"""

df_migr = pd.read_sql(query_migration, conn)
print(df_migr.head())
print(df_migr["geo_code"].value_counts())


df_de = df_pop[df_pop["geo_code"] == "DE"].copy()

plt.figure()
plt.plot(df_de["year"], df_de["population_value"], marker="o")
plt.xlabel("Year")
plt.ylabel("Population (DE)")
plt.title("Population of Germany over time")
plt.grid(True)
plt.show()

df_de


# feature = year, target = population_value
X = df_de[["year"]].values  # 2D
y = df_de["population_value"].values  # 1D

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Vitet:", X.flatten())
print("Popullsia:", y)


# marrim të gjitha vitet përveç 2 të fundit për train
X_train = X[:-2]
y_train = y[:-2]

# 2 vitet e fundit për test
X_test = X[-2:]
y_test = y[-2:]

print("Train years:", X_train.flatten())
print("Test years:", X_test.flatten())

model1 = LinearRegression()
model1.fit(X_train, y_train)

y_pred_test = model1.predict(X_test)

mae1 = mean_absolute_error(y_test, y_pred_test)
rmse1 = mean_squared_error(y_test, y_pred_test, squared=False)

print("MAE (model1):", mae1)
print("RMSE (model1):", rmse1)
print("Krahasim (year, real, pred):")
for year, real_v, pred_v in zip(X_test.flatten(), y_test, y_pred_test):
    print(year, int(real_v), int(pred_v))


y_all_pred = model1.predict(X)

plt.figure()
plt.plot(df_de["year"], y, label="Real", marker="o")
plt.plot(df_de["year"], y_all_pred, label="Predicted (model1)", linestyle="--", marker="x")
plt.xlabel("Year")
plt.ylabel("Population (DE)")
plt.title("Real vs Predicted population (Germany) - Model1 (year only)")
plt.legend()
plt.grid(True)
plt.show()


last_year = int(df_de["year"].max())
future_years = np.array([[y] for y in range(last_year + 1, last_year + 6)])

print("Vitet e ardhshme:", future_years.flatten())

future_pred = model1.predict(future_years)

print("Parashikimet (Model1, vetëm vit):")
for year, pred in zip(future_years.flatten(), future_pred):
    print(year, int(pred))

plt.figure()
plt.plot(df_de["year"], df_de["population_value"], label="Real", marker="o")
plt.plot(future_years.flatten(), future_pred, label="Forecast (Model1)", marker="s", linestyle="--")
plt.xlabel("Year")
plt.ylabel("Population (DE)")
plt.title("Population forecast for Germany (next 5 years) - Model1")
plt.legend()
plt.grid(True)
plt.show()


# filtro migracionin për DE
df_migr_de = df_migr[df_migr["geo_code"] == "DE"].copy()

# agregim sipas vitit (total_migration = suma e 'value' për të gjitha citizenship)
df_migr_agg = (
    df_migr_de
    .groupby("year", as_index=False)["value"]
    .sum()
    .rename(columns={"value": "total_migration"})
)

print(df_migr_agg.head())

# bashko popullsinë me migracionin sipas vitit
df_de_full = df_de.merge(df_migr_agg, on="year", how="left")

print(df_de_full.head())
print(df_de_full.isna().sum())  # shih nëse ka NaN


# X2 = [year, total_migration], zëvendëso NaN me 0
X2 = df_de_full[["year", "total_migration"]].fillna(0).values
y2 = df_de_full["population_value"].values

X2_train = X2[:-2]
y2_train = y2[:-2]
X2_test = X2[-2:]
y2_test = y2[-2:]

print("X2_train shape:", X2_train.shape)
print("X2_test shape:", X2_test.shape)

model2 = LinearRegression()
model2.fit(X2_train, y2_train)

y2_pred_test = model2.predict(X2_test)

mae2 = mean_absolute_error(y2_test, y2_pred_test)
rmse2 = mean_squared_error(y2_test, y2_pred_test, squared=False)

print("MAE (model2, year + migration):", mae2)
print("RMSE (model2):", rmse2)
print("Krahasim (year, real, pred):")
for row, real_v, pred_v in zip(X2_test, y2_test, y2_pred_test):
    print(int(row[0]), int(real_v), int(pred_v))


# llogarisim mesataren historike të total_migration për DE
avg_migration = df_de_full["total_migration"].mean()
print("Mesatarja historike e total_migration (DE):", avg_migration)

last_year = int(df_de_full["year"].max())
future_years2 = list(range(last_year + 1, last_year + 6))

# ndërto X_future2 me vit + migracion mesatar
X_future2 = np.array([[year, avg_migration] for year in future_years2])
future_pred2 = model2.predict(X_future2)

print("Parashikimet (Model2, year + avg migration):")
for year, pred in zip(future_years2, future_pred2):
    print(year, int(pred))

plt.figure()
plt.plot(df_de_full["year"], df_de_full["population_value"], label="Real", marker="o")
plt.plot(future_years2, future_pred2, label="Forecast (Model2)", marker="s", linestyle="--")
plt.xlabel("Year")
plt.ylabel("Population (DE)")
plt.title("Population forecast for Germany (next 5 years) - Model2")
plt.legend()
plt.grid(True)
plt.show()


models_dir = Path("../models")
models_dir.mkdir(exist_ok=True)

model_path = models_dir / "de_population_model.pkl"
joblib.dump(model2, model_path)

print("Modeli u ruajt te:", model_path.resolve())


loaded_model = joblib.load(model_path)

# shembull: vit i ardhshëm + migracion hipotetik 200000
test_input = np.array([[2030, 200000]])
prediction = loaded_model.predict(test_input)[0]

print("Parashikim për vitin 2030 me 200000 migration:", int(prediction))


